In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import mpld3
import numpy as np
import pandas as pd
import csv
from mpld3 import plugins

mpld3.enable_notebook()

In [2]:
# Define CSS to control our custom labels# Define 
css = """
table
{
  border-collapse: collapse;
}
th
{
  color: #ffffff;
  background-color: #000000;
}
td
{
  background-color: #cccccc;
}
th, td {
  padding: 3px;
}
table, th, td
{
  font-family:Arial, Helvetica, sans-serif;
  border: 1px solid black;
  text-align: right;
}
"""

### Loading Data

In [3]:
df = pd.read_csv("AllBirdsv4.csv")
df_test = pd.read_csv("Test Birds Location.csv")

### Pre-processing

#### All Birds file

In [4]:
# replace "?" characters
df = df.replace('?', None)
df["Y"] = df["Y"].map(lambda y: y.replace("?",""))
df["Y"] = df["Y"].astype('int64')

# scale coordinates for our map (1000*1000)
df["X"] = df["X"].map(lambda x: x * 5)
df["Y"] = df["Y"].map(lambda y: y * 5)

species_list = df["English_name"].unique()
species_index = []
for element in df["English_name"]:
    species_index.append(species_list.tolist().index(element))
df["Birds index"] = species_index

# get years from date column
df["Year"] = df["Date"]
for i in range(df.shape[0]):
    date = df["Year"][i]
    if len(date.split("/")) == 3:
        date = date.split("/")[2]
        df.at[i, "Year"] = date
    elif len(date.split("-")) == 3 and date.split("-")[0] != '0000':
        date = date.split("-")[0]
        df.at[i, "Year"] = date
    else:
        df.at[i, "Year"] = None

In [5]:
print(df["Year"].unique())

['2018' '2017' '2008' '2016' '2000' '2015' '2014' '2013' '1993' '2012'
 '2009' '2004' '2005' '1996' '2010' '2007' '2011' '1991' '2003' '1998'
 '1990' '1992' '1997' '2006' '2002' None '1999' '1988' '1986' '1994'
 '1989' '1983' '1995' '2001']


#### Test Birds file

In [6]:
# rename columns of X and Y
df_test.columns = ["ID", "X", "Y"]

# scale coordinates for our map (1000*1000)
df_test["X"] = df_test["X"].map(lambda x: x * 5)
df_test["Y"] = df_test["Y"].map(lambda y: y * 5)

In [7]:
img = mpimg.imread('map.jpg')
img = img[::-1] # reverse order for y-axis so that (0,0) is bottom-left

fig = plt.figure(figsize=(10,10))

map_plot = fig.add_subplot()
plt.imshow(img, origin='lower')
plt.title('Birds locations on Lekagul Roadways map', size=20)
plt.xlabel('x')
plt.ylabel('y')
cmap = plt.get_cmap('tab20')

# plot all birds
scatter_all_birds = plt.scatter(list(df["X"]), list(df["Y"]), s=60, alpha=0.5, c=cmap(df["Birds index"]))


# plot the 15 "test birds"
scatter_test_birds = plt.scatter(list(df_test["X"]), list(df_test["Y"]), s=300, c='red', marker='^')

# create custom labels
labels_test_birds = []
for i in range(df_test.shape[0]):
    label = '<table>' 
    label += '<tr><th></th><th>Test Bird {}</th></tr>'.format(df_test["ID"][i])
    label += '<tr><th>X</th><td>{}</td></tr>'.format(df_test["X"][i])
    label += '<tr><th>Y</th><td>{}</td></tr>'.format(df_test["Y"][i])
    label += '</table>'
    labels_test_birds.append(label)
    
labels_all_birds = []
for i in range(df.shape[0]):
    label = '<table>'
    label += '<tr><th></th><th>{}</th></tr>'.format(df["English_name"][i])
    label += '<tr><th>X</th><td>{}</td></tr>'.format(df["X"][i])
    label += '<tr><th>Y</th><td>{}</td></tr>'.format(df["Y"][i])
    label += '</table>'
    labels_all_birds.append(label)

# tooltip shown on mouse hover over data points
tooltip_test_birds = plugins.PointHTMLTooltip(scatter_test_birds, labels=labels_test_birds, css=css)
tooltip_all_birds = plugins.PointHTMLTooltip(scatter_all_birds, labels=labels_all_birds, css=css)
plugins.connect(fig, tooltip_test_birds)
plugins.connect(fig, tooltip_all_birds)

mpld3.display(fig)